<a href="https://colab.research.google.com/github/a-tuck-ai/AEI4/blob/main/Anna_T_Advanced_Retrieval_with_LangChain_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [42]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.7 which is incompatible.


We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [43]:
!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [44]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


In [45]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key:··········


## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [46]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2024-10-01 18:25:16--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0s      

2024-10-01 18:25:16 (65.0 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2024-10-01 18:25:16--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [47]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [48]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 28, 18, 25, 17, 100213)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [49]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [50]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [51]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [52]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [53]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [54]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews and high ratings given by critics and viewers.'

In [55]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review_Title: "A Masterpiece & Brilliant Sequel"\n- Review_Url: \'/review/rw4854296/?ref_=tt_urv\''

In [56]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance on the gangsters who killed his dog and took everything from him. The intense and action-packed story follows John Wick as he unleashes a maelstrom of destruction against those who crossed him.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [57]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [58]:
naive_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [59]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Opinions on John Wick seem to vary. Some people really enjoyed it and praised its action sequences and style, while others found it lacking in plot and substance. It ultimately depends on personal preference.'

In [60]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I don't know."

In [61]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, John Wick, is a retired hitman seeking vengeance for the death of his dog, which was a gift from his deceased wife. This leads to a series of intense action sequences as he hunts down those responsible.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [62]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [63]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [64]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [65]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review Title: A Masterpiece & Brilliant Sequel\n- Review URL: '/review/rw4854296/?ref_=tt_urv'"

In [66]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving his issues with the Russian mafia, John Wick is forced back into action when a mobster named Santino D'Antonio asks him to kill his sister in Rome. Wick completes the task but then becomes the target of a seven-million dollar contract put on him by Santino. Wick seeks revenge against Santino."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [67]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [68]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [69]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The film was praised for its slickness, brilliant action sequences, Keanu Reeves' performance, and overall entertainment value. Reviewers highlighted the intense action, stylish stunts, and the fun factor of the movie."

In [70]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review URL: '/review/rw4854296/?ref_=tt_urv'"

In [71]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," the story begins with the protagonist, John, losing his wife. Subsequently, a young Russian-American punk attacks John, leading to a series of events where John\'s dog is killed, his car is stolen, and his house is destroyed. It is revealed that John is a super-assassin seeking revenge on the mobster responsible for these actions. The film revolves around a battle between John Wick and the Russian mobster and his gang.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [72]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [73]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

<ipython-input-73-4887b10d4438>:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [74]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [75]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [76]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [77]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'It seems that opinions on John Wick vary among viewers. Some people really enjoy the series, while others may not like it as much.'

In [78]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [79]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, played by Keanu Reeves, is a retired assassin who comes out of retirement after his dog is killed and his car is stolen. He embarks on a violent journey of revenge, taking down numerous assassins in Italy, Canada, and Manhattan.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [80]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [81]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [82]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews and high ratings given by the reviewers.'

In [83]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review: "/review/rw4854296/?ref_=tt_urv"'

In [84]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, an ex-hitman comes out of retirement to seek vengeance on the gangsters who killed his dog and took everything from him. The story is filled with violent action, shootouts, and breathtaking fights as John Wick unleashes a maelstrom of destruction against those who come after him. It's a story of revenge and survival in a world where every action has consequences."

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [85]:
!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [86]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [87]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [88]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [89]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [90]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [91]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, it seems that people generally liked John Wick. The reviews highlight the cool action sequences, the sleek style, and the engaging storyline.'

In [92]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [93]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story follows his journey as he unleashes a wave of destruction against those who wronged him.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [149]:
!pip install -U langchain==0.2.16 langchain-community==0.2.16 langchain-core==0.2.38 langchain-experimental==0.0.65 langchain-openai==0.1.23 langchain-qdrant==0.1.4 qdrant-client==1.11.3 ragas==0.1.20 rank-bm25==0.2.2 langchain_cohere

  Using cached langchain_cohere-0.3.0-py3-none-any.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.


In [150]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")


Enter your OpenAI API Key:··········


In [151]:


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")


LangChain API Key:··········


In [152]:
from uuid import uuid4

project_name = f"AIM: Advanced Retrieval w/ Lanchain - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_PROJECT"] = project_name
print(f"LangChain project set to: {project_name}")

LangChain project set to: AIM: Advanced Retrieval w/ Lanchain - bc846c8f


In [153]:
from langchain_community.document_loaders import CSVLoader

try:
    test_documents = CSVLoader("john_wick_1.csv").load()
    print(f"Loaded {len(test_documents)} documents from john_wick_1.csv")
except FileNotFoundError:
    print("Error: john_wick_1.csv file not found.")
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")

Loaded 25 documents from john_wick_1.csv


In [154]:
from ragas.testset.generator import TestsetGenerator
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.evolutions import simple, reasoning, multi_context

# GPT-3.5-turbo for initial generation due to its good balance of performance and cost-effectiveness
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")

# GPT-4 for critique to ensure high-quality refinement of questions and answers
# The newer version (0125-preview) is used for potentially improved performance
critic_llm = ChatOpenAI(model="gpt-4-0125-preview", tags=["base_llm"])

# OpenAI's text embeddings for semantic understanding and representation of text
embeddings = OpenAIEmbeddings()

# Creating the TestsetGenerator using the defined language models and embeddings
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Distribution of question types:
# - Simple questions for basic comprehension and fact-checking
# - Multi-context questions to test information synthesis across multiple sources
# - Reasoning questions for higher-order thinking and complex problem-solving
distributions = {
    simple: 0.6,       # 60% simple questions for broad coverage of the dataset
    multi_context: 0.3,# 30% multi-context for testing information integration skills
    reasoning: 0.1     # 10% reasoning for challenging, thought-provoking questions
}

In [155]:
# Generate the test set
result = generator.generate_with_langchain_docs(test_documents, 20, distributions, with_debugging_logs=False)



embedding nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating:   0%|          | 0/20 [00:00<?, ?it/s]

In [156]:
from collections import Counter

type_counts = Counter(item.evolution_type for item in result.test_data)
print("Question type distribution:")
for type, count in type_counts.items():
    print(f"{type}: {count} ({count/len(result.test_data)*100:.2f}%)")

Question type distribution:
simple: 12 (75.00%)
multi_context: 3 (18.75%)
reasoning: 1 (6.25%)


In [157]:

from datasets import Dataset

dataset = Dataset.from_pandas(result.to_pandas())


In [158]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
)

In [159]:
# Define metric sets for each retriever
naive_metrics = [context_precision, context_recall, answer_relevancy]
bm25_metrics = [context_precision, context_recall, answer_relevancy]
multi_query_metrics = [faithfulness, answer_relevancy, context_recall, context_precision, answer_correctness]
parent_document_metrics = [faithfulness, answer_relevancy, context_recall, context_precision, answer_correctness]
#compression_metrics = [faithfulness, answer_relevancy, context_recall, context_precision, answer_correctness]
#ensemble_metrics = [faithfulness, answer_relevancy, context_recall, context_precision, answer_correctness]

In [160]:
def answer_question(chain, dataset):
    """
    Process a dataset of questions through a retrieval-generation chain and format the results.

    Args:
    chain: A retrieval-generation chain that can process questions.
    dataset: A Hugging Face Dataset containing questions and ground truths.

    Returns:
    A new Dataset with questions, generated answers, retrieved contexts, and ground truths.
    """
    # Extract questions and ground truths from the dataset
    df = dataset.to_pandas()
    questions = df["question"].tolist()
    ground_truths = df["ground_truth"].tolist()

    answers = []
    contexts = []

    for question in questions:
        # Process each question through the chain
        result = chain.invoke({"question": question})
        # Handle both string and object responses
        if isinstance(result["response"], str):
            answers.append(result["response"])
        else:
            answers.append(result["response"].content)
        contexts.append([context.page_content for context in result["context"]])

    # Create and return a new Dataset with all information
    return Dataset.from_dict({
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths,
    })

In [161]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

# Create a Qdrant client (this will create an in-memory database)
client = QdrantClient(":memory:")

# Create the vector store
vectorstore = QdrantVectorStore.from_documents(
    documents,  # Changed from test_documents to documents
    embeddings,
    location=":memory:",
    collection_name="john_wick_reviews",
)

# Now you can create your naive retriever
naive_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [162]:
naive_retrieval_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model | StrOutputParser(), "context": itemgetter("context")}
)

In [163]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

chat_model = ChatOpenAI()

# Define the RAG prompt template
rag_prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

Context: {context}

Question: {question}

Answer: """)

In [164]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import EnsembleRetriever

# BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(documents)

# Multi-Query Retriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

# Parent Document Retriever
parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)
parent_document_retriever.add_documents(parent_docs, ids=None)

# Contextual Compression Retriever and Ensemble Retriever are commented out because I was hitting token limits.

In [165]:
# Create chains for each retriever
retriever_chains = {
    "Naive": naive_retrieval_chain,
    "BM25": (
        {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | chat_model | StrOutputParser(), "context": itemgetter("context")}
    ),
    "Multi-Query": (
        {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | chat_model | StrOutputParser(), "context": itemgetter("context")}
    ),
    "Parent Document": (
        {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | chat_model | StrOutputParser(), "context": itemgetter("context")}
    )
}

# Print the keys of retriever_chains to confirm all chains are created
print("Created chains for:", list(retriever_chains.keys()))

Created chains for: ['Naive', 'BM25', 'Multi-Query', 'Parent Document']


In [166]:
from ragas import evaluate

# Naive Retrieval
naive_dataset = answer_question(naive_retrieval_chain, dataset)
naive_results = evaluate(naive_dataset, naive_metrics)
print("Naive Retrieval Results:", naive_results)


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Naive Retrieval Results: {'context_precision': 0.6817, 'context_recall': 0.8125, 'answer_relevancy': 0.9052}


In [167]:

# BM25 Retrieval
bm25_dataset = answer_question(retriever_chains["BM25"], dataset)
bm25_results = evaluate(bm25_dataset, bm25_metrics)
print("BM25 Retrieval Results:", bm25_results)


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

BM25 Retrieval Results: {'context_precision': 0.7222, 'context_recall': 0.8229, 'answer_relevancy': 0.9715}


In [168]:

# Multi-Query Retrieval
multi_query_dataset = answer_question(retriever_chains["Multi-Query"], dataset)
multi_query_results = evaluate(multi_query_dataset, multi_query_metrics)
print("Multi-Query Retrieval Results:", multi_query_results)


Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

Multi-Query Retrieval Results: {'faithfulness': 0.9244, 'answer_relevancy': 0.9695, 'context_recall': 0.9688, 'context_precision': 0.5948, 'answer_correctness': 0.6837}


In [169]:

# Parent Document Retrieval
parent_document_dataset = answer_question(retriever_chains["Parent Document"], dataset)
parent_document_results = evaluate(parent_document_dataset, parent_document_metrics)
print("Parent Document Retrieval Results:", parent_document_results)


Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

Parent Document Retrieval Results: {'faithfulness': 0.7952, 'answer_relevancy': 0.9702, 'context_recall': 0.8438, 'context_precision': 0.8021, 'answer_correctness': 0.7096}


In [170]:

# Contextual Compression Retrieval - would have liked to run this but token limits!!


In [171]:

# Ensemble Retrieval - would have liked to run this but token limits!!



In [172]:
import pandas as pd

# Create DataFrames for each result
df_naive = pd.DataFrame(list(naive_results.items()), columns=['Metric', 'Naive'])
df_bm25 = pd.DataFrame(list(bm25_results.items()), columns=['Metric', 'BM25'])
df_multi_query = pd.DataFrame(list(multi_query_results.items()), columns=['Metric', 'MultiQuery'])
df_parent_document = pd.DataFrame(list(parent_document_results.items()), columns=['Metric', 'ParentDocument'])
#df_compression = pd.DataFrame(list(compression_results.items()), columns=['Metric', 'Compression'])
#df_ensemble = pd.DataFrame(list(ensemble_results.items()), columns=['Metric', 'Ensemble'])

# Merge all DataFrames
df_merged = df_naive.merge(df_bm25, on='Metric')
df_merged = df_merged.merge(df_multi_query, on='Metric')
df_merged = df_merged.merge(df_parent_document, on='Metric')
#df_merged = df_merged.merge(df_compression, on='Metric')
#df_merged = df_merged.merge(df_ensemble, on='Metric')

print(df_merged)

              Metric     Naive      BM25  MultiQuery  ParentDocument
0  context_precision  0.681693  0.722222    0.594770        0.802083
1     context_recall  0.812500  0.822917    0.968750        0.843750
2   answer_relevancy  0.905170  0.971517    0.969521        0.970216


In [173]:
import time
import os
from langsmith import Client

client = Client()

def run_with_langsmith(chain, dataset, name):
    start_time = time.time()
    result = answer_question(chain, dataset)
    end_time = time.time()
    latency = end_time - start_time

    try:
        # Get the run from LangSmith
        runs = client.list_runs(
            project_name=os.environ.get("LANGCHAIN_PROJECT", "default"),
            execution_order=1,
            error=False,
        )
        run = next(runs)

        tokens = run.prompt_tokens + run.completion_tokens
        cost = run.total_tokens * 0.002 / 1000  # Assuming $0.002 per 1K tokens

        print(f"{name} - Latency: {latency:.2f}s, Tokens: {tokens}, Cost: ${cost:.6f}")
    except Exception as e:
        print(f"Error accessing LangSmith: {e}")
        print(f"{name} - Latency: {latency:.2f}s")

    return result

# Use this function for each evaluation
print("Running evaluations with LangSmith (if available)...")
naive_dataset = run_with_langsmith(naive_retrieval_chain, dataset, "Naive Retrieval")
bm25_dataset = run_with_langsmith(retriever_chains["BM25"], dataset, "BM25 Retrieval")
multi_query_dataset = run_with_langsmith(retriever_chains["Multi-Query"], dataset, "Multi-Query Retrieval")
parent_document_dataset = run_with_langsmith(retriever_chains["Parent Document"], dataset, "Parent Document Retrieval")

Running evaluations with LangSmith (if available)...
Error accessing LangSmith: Project AIM: Advanced Retrieval w/ Lanchain - bc846c8f not found
Naive Retrieval - Latency: 27.70s
Error accessing LangSmith: Project AIM: Advanced Retrieval w/ Lanchain - bc846c8f not found
BM25 Retrieval - Latency: 19.99s
Error accessing LangSmith: Project AIM: Advanced Retrieval w/ Lanchain - bc846c8f not found
Multi-Query Retrieval - Latency: 53.02s
Error accessing LangSmith: Project AIM: Advanced Retrieval w/ Lanchain - bc846c8f not found
Parent Document Retrieval - Latency: 21.94s


In [175]:
# Analyze the results
best_performer = df_merged.iloc[:, 1:].mean().idxmax()
print(f"The best overall performer is: {best_performer}")

# Add your detailed analysis here, considering:
# - Performance across different metrics
# - Latency
# - Token usage and cost
# - Suitability for the John Wick dataset

The best overall performer is: ParentDocument


The evaluation of four retrieval methods for the John Wick movie dataset reveals that the Parent Document Retriever is the most effective overall. It achieves the highest context precision (0.802083) and maintains excellent scores in context recall (0.843750) and answer relevancy (0.970216), while keeping a low latency of 21.94 seconds. This balanced performance indicates its ability to capture nuanced context in movie reviews while providing highly relevant answers efficiently. BM25, despite having the highest answer relevancy (0.971517) and lowest latency (19.99s), falls short in context precision. Multi-Query Retrieval excels in recall (0.968750) but suffers from low precision and high latency (53.02s), making it less practical for real-time applications. The Naive approach, while simple, is outperformed across all metrics. The Parent Document Retriever's success likely stems from its ability to maintain the hierarchical structure of movie reviews, preserving both specific details and broader context. This makes it particularly well-suited for diverse queries about movies, offering the best compromise between performance, speed, and contextual relevance for the John Wick dataset.